In [32]:
import os, sys
import re
import json
import glob
import psycopg2
from psycopg2 import sql
from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT
from sqlalchemy import create_engine

In [35]:
rpath = os.path.abspath('..')
if rpath not in sys.path:
    sys.path.insert(0, rpath)

from src.loader import SlackDataLoader
import src.utils as utils

## This is for creating a database uisng psycopg2

In [15]:
# password and user shouldn't be available in public this is just for practice
# Todo: don't create databse if it exists
db_params = {
    'host': '127.0.0.1',
    'user': 'postgres',
    'password': '123',
    'port': '5467'
}
connection = psycopg2.connect(**db_params)
connection.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT)
cursor = connection.cursor()
db_name = 'week_0'
quoted_db_name = sql.Identifier(db_name)
create_db_query = sql.SQL("CREATE DATABASE {}").format(quoted_db_name)
cursor.execute(create_db_query)

## This is for creating a table using schema file using psycopg2

In [24]:
# Todo: the schema file need to be updated to reflect the whole structure
# few column are choosen for simplicity

In [33]:
db_params = {
    'host': '127.0.0.1',
    'user': 'postgres',
    'password': '123',
    'port': '5467',
    'database': 'week_0'
}

In [20]:
connection = psycopg2.connect(**db_params)
connection.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT)

In [21]:
cursor = connection.cursor()

In [22]:
schema_file_path = 'schema.sql'

In [23]:
with open(schema_file_path, 'r') as schema_file:
    schema_sql = schema_file.read()

In [24]:
cursor.execute(schema_sql)
print("Table created successfully.")

# Close the cursor and connection
cursor.close()
connection.close()

Table created successfully.


## Let's add data to the table using sqlalchemy and pandas

In [34]:
engine = create_engine(f"postgresql+psycopg2://{db_params['user']}:{db_params['password']}@{db_params['host']}:{db_params['port']}/{db_params['database']}")

In [37]:
# Please include the path to datafiles here for this code to work
read_file = SlackDataLoader("../anonymized")
message_with_user = read_file.slack_parser("../anonymized/Week8and9/")
selected_columns = ['msg_type', 'sender_name', 'msg_dist_type', 'reply_count']
df_selected = message_with_user[selected_columns]
table_name = "messages"
df_selected.to_sql(table_name, engine, index=False, if_exists='replace')
print(f"\nData saved to table '{table_name}' in the database.")


Data saved to table 'messages' in the database.
